## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,80.76,89,75,10.36,light rain
1,1,Punta Arenas,CL,-53.1500,-70.9167,50.11,57,90,11.50,overcast clouds
2,2,Saint-Philippe,RE,-21.3585,55.7679,81.95,81,94,3.67,light rain
3,3,Jamestown,US,42.0970,-79.2353,31.32,93,90,6.91,light snow
4,4,Nizhniy Kuranakh,RU,58.8353,125.4869,2.66,90,99,6.87,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,80.76,89,75,10.36,light rain
2,2,Saint-Philippe,RE,-21.3585,55.7679,81.95,81,94,3.67,light rain
16,16,Matara,LK,5.9485,80.5353,82.02,75,93,3.04,overcast clouds
19,19,Rikitea,PF,-23.1203,-134.9692,76.39,79,74,16.40,broken clouds
21,21,Kapaa,US,22.0752,-159.3190,75.18,73,1,8.05,clear sky
28,28,Pitimbu,BR,-7.4706,-34.8086,75.79,70,80,9.26,broken clouds
29,29,Avarua,CK,-21.2078,-159.7750,75.25,83,75,3.44,broken clouds
34,34,Kota Kinabalu,MY,5.9749,116.0724,89.58,70,20,4.61,few clouds
38,38,Bathsheba,BB,13.2167,-59.5167,81.21,78,40,13.80,scattered clouds
39,39,Salalah,OM,17.0151,54.0924,82.49,30,0,4.61,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
#finding null values
preferred_cities_df.count()

City_ID                180
City                   180
Country                180
Lat                    180
Lng                    180
Max Temp               180
Humidity               180
Cloudiness             180
Wind Speed             180
Current Description    180
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#finding null values
clean_df=preferred_cities_df.dropna()

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,80.76,light rain,-21.2000,-175.2000,
2,Saint-Philippe,RE,81.95,light rain,-21.3585,55.7679,
16,Matara,LK,82.02,overcast clouds,5.9485,80.5353,
19,Rikitea,PF,76.39,broken clouds,-23.1203,-134.9692,
21,Kapaa,US,75.18,clear sky,22.0752,-159.3190,
28,Pitimbu,BR,75.79,broken clouds,-7.4706,-34.8086,
29,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,
34,Kota Kinabalu,MY,89.58,few clouds,5.9749,116.0724,
38,Bathsheba,BB,81.21,scattered clouds,13.2167,-59.5167,
39,Salalah,OM,82.49,clear sky,17.0151,54.0924,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"    

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    hotels = requests.get(base_url, params=params).json()    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,80.76,light rain,-21.2000,-175.2000,Keleti Beach Resort
2,Saint-Philippe,RE,81.95,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
16,Matara,LK,82.02,overcast clouds,5.9485,80.5353,Amaloh Boutique Resort
19,Rikitea,PF,76.39,broken clouds,-23.1203,-134.9692,People ThankYou
21,Kapaa,US,75.18,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
701,Lamu,KE,80.82,moderate rain,-2.2717,40.9020,Lamu House
704,Itarema,BR,80.08,broken clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
707,Puerto Morazan,NI,76.21,overcast clouds,12.8504,-87.1717,Iglesia de Dios de la profecia
709,Labuhan,ID,81.00,light rain,-6.8844,112.2051,PT PPI Lamongan Tambak C


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))